### Librerías necesarias y lectura de datos

In [1]:
#Librerias
import pandas as pd
import matplotlib.pyplot as plt
from networkx.algorithms.bipartite.basic import color

# Columnas que se tomarán en cuenta para el análisis
columnas = ["Nomestado","Nommunicipio","Nomcultivo","Volumenproduccion", "Sembrada","Nomcicloproductivo","Nommodalidad","Nomunidad", "Cosechada", "Siniestrada", "Rendimiento", "Preciomediorural", "Valorproduccion"]

# Lectura del archivo csv en un dataframe
df = pd.read_csv("Cierre_agr_mun_2023.csv", encoding="latin1")

# Asignación de columnas de interés
df = df[columnas]

# Borrar notacion cientifica
pd.set_option('display.float_format', '{:.0f}'.format)

# Formatear numeros para aumentar legibilidad
pd.options.display.float_format = '{:,.0f}'.format


In [2]:
df.head

<bound method NDFrame.head of             Nomestado    Nommunicipio                    Nomcultivo  \
0      Aguascalientes  Aguascalientes      Avena forrajera en verde   
1      Aguascalientes  Aguascalientes                       Brócoli   
2      Aguascalientes  Aguascalientes     Cebada forrajera en verde   
3      Aguascalientes  Aguascalientes  Triticale forrajero en verde   
4      Aguascalientes  Aguascalientes             Pastos y praderas   
...               ...             ...                           ...   
38691       Zacatecas           Jalpa                          Nuez   
38692       Zacatecas           Jalpa                           Uva   
38693       Zacatecas           Jalpa             Pastos y praderas   
38694       Zacatecas           Jalpa                         Agave   
38695       Zacatecas           Jalpa             Pastos y praderas   

       Volumenproduccion  Sembrada Nomcicloproductivo Nommodalidad Nomunidad  \
0                 37,170     1,400   

## Conteo y selección de cultivos a nivel nacional
Se encontró un total de 317 cultivos diferentes

In [3]:

df["Nomcultivo"].value_counts().sort_index()


Nomcultivo
Aceituna                      22
Acelga                        78
Achiote                       21
Agapando (gruesa)              7
Agave                        452
                            ... 
Zempoalxochitl               124
Zempoalxochitl (gruesa)        1
Zempoalxochitl (manojo)       14
Zempoalxochitl (planta)        4
Árbol de navidad (planta)     30
Name: count, Length: 317, dtype: int64

#### `Limón`
Dentro de los 317 tipos diferentes tipos de cultivos se eligió el `Limón` porque es el que representa la mayor cantidad de producción y se cultivó en 28 estados del país. Los siguientes cultivos serán descartados.
- `Lima`: Se cultivó en 13 estados del país y Morelos no se encuentra en este grupo
- `Limón real`: Se cultivó en 1 estado del país (Sonora)



#### Estados del país en donde se cultiva la Lima

In [4]:
lima = df.loc[df["Nomcultivo"]=="Lima"]
lima["Nomestado"].value_counts().sort_index()

Nomestado
Aguascalientes      1
Baja California     1
Colima              2
Durango             5
Hidalgo             1
Jalisco            15
Michoacán           7
México              5
Nayarit             4
Puebla              7
Tamaulipas          2
Yucatán             3
Zacatecas           3
Name: count, dtype: int64

#### Estados del país en donde se cultiva el Limón real

In [5]:

limon_real = df.loc[df["Nomcultivo"]=="Limón real"]
limon_real["Nomestado"].value_counts()

Nomestado
Sonora    1
Name: count, dtype: int64

#### Estados del país en donde se cultiva el Limón

Se desconoce la especie que se tomó en cuenta para este tipo de cultivo, ya que en México se producen 2 especies de limón:
- `Limón persa` (Citrus × latifolia)
- `Limón mexicano`(Citrus × aurantifolia)

In [6]:
limon = df.loc[df["Nomcultivo"]=="Limón"]
limon["Nomestado"].value_counts()

Nomestado
Veracruz               99
Yucatán                65
Oaxaca                 65
Jalisco                64
Guerrero               63
Chiapas                58
Michoacán              34
Sinaloa                29
Nayarit                29
Puebla                 25
Morelos                23
Tamaulipas             23
México                 21
Hidalgo                18
Campeche               17
Quintana Roo           13
Colima                 12
Sonora                 12
Guanajuato             11
Tabasco                11
Baja California        10
Zacatecas               7
Durango                 5
Baja California Sur     4
San Luis Potosí         3
Nuevo León              3
Querétaro               2
Aguascalientes          1
Name: count, dtype: int64

# Estadísticas a nivel nacional


 - `Volumenproduccion`: representa el volumen de producción de la superficie cosechada cuya unidad de medida son las toneladas, con excepción del maguey pulquero y trigo ornamental, con una métrica de miles de litros y gruesas, respectivamente (información tomada del diccionario de datos).

In [7]:
# Agrupar y sumar directamente sin seleccionar una columna específica
prod_nac_df = limon.groupby("Nomestado", as_index=False)["Volumenproduccion"].sum()

# Ordenar los resultados
prod_nac_df = prod_nac_df.sort_values(by="Volumenproduccion", ascending=True).round(2)

prod_nac_df


,Nomestado,Volumenproduccion
18,Querétaro,30
2,Baja California Sur,254
6,Durango,526
7,Guanajuato,970
13,México,"1,180"
9,Hidalgo,"1,463"
1,Baja California,"2,692"
12,Morelos,"4,807"
15,Nuevo León,"5,170"
0,Aguascalientes,"5,271"


In [8]:
prod_nac_df["Porcentaje"]=((prod_nac_df["Volumenproduccion"]/prod_nac_df["Volumenproduccion"].sum())*100).round(2)
prod_nac_df

,Nomestado,Volumenproduccion,Porcentaje
18,Querétaro,30,0
2,Baja California Sur,254,0
6,Durango,526,0
7,Guanajuato,970,0
13,México,"1,180",0
9,Hidalgo,"1,463",0
1,Baja California,"2,692",0
12,Morelos,"4,807",0
15,Nuevo León,"5,170",0
0,Aguascalientes,"5,271",0


In [54]:
import plotly.express as px


prod_nac_df = prod_nac_df.rename(columns={
    'Nomestado': 'Estado',
    'Volumenproduccion': 'Volumen de Producción'
})

# Crear el gráfico de barras
fig = px.bar(
    prod_nac_df,
    x='Estado',
    y='Volumen de Producción',
    hover_data={'Porcentaje': ':.2f'},
    text_auto='.2s',
    color="Volumen de Producción",

    title="Producción de Limón por Estado",


    color_continuous_scale=px.colors.sequential.Viridis
)

# Actualizar el layout
fig.update_layout(
    title="Producción de Limón por Estado",
     title_font=dict(
        family="Arial, sans-serif",  # Fuente estética, puedes usar otras como 'Courier New', 'Times New Roman', etc.
        size=24,                    # Tamaño del título
        color="darkblue"            # Color del título
    ),
    xaxis_title="Estado",  # Ahora es el eje X
    yaxis_title="Volumen de Producción (toneladas)",  # Ahora es el eje Y

    font=dict(size=8),

    yaxis=dict(
        tickformat=",",           # Formato de los ticks

 ),
    template="plotly_white",

)


fig.update_traces(textfont_size=5, textangle=0, textposition="outside", cliponaxis=False)

# Mostrar el gráfico
fig.show()




# Guardar el gráfico como SVG

fig.write_image("produccion_nacional.svg", engine="kaleido")
# Save the graph as an HTML file
fig.write_html("produccion_nacional.html")

# Optionally, open the HTML file in the default web browser
import webbrowser
webbrowser.open("produccion_nacional.html")



True

# Estadísticas a nivel estatal

In [10]:

prod_est_df= limon.loc[limon["Nomestado"]=="Morelos"]

In [11]:
prod_est_df = prod_est_df.groupby("Nommunicipio", as_index=False)["Volumenproduccion"].sum()

# Ordenar los resultados
prod_est_df = prod_est_df.sort_values(by="Volumenproduccion", ascending=True)

prod_est_df

,Nommunicipio,Volumenproduccion
17,Tlaquiltenango,25
7,Jiutepec,32
1,Atlatlahucan,46
13,Temixco,58
21,Yecapixtla,62
19,Xochitepec,65
20,Yautepec,113
9,Jonacatepec,114
11,Miacatlán,119
8,Jojutla,122


In [12]:
prod_est_df["Porcentaje"]=((prod_est_df["Volumenproduccion"]/prod_est_df["Volumenproduccion"].sum())*100).round(2)
prod_est_df

,Nommunicipio,Volumenproduccion,Porcentaje
17,Tlaquiltenango,25,1
7,Jiutepec,32,1
1,Atlatlahucan,46,1
13,Temixco,58,1
21,Yecapixtla,62,1
19,Xochitepec,65,1
20,Yautepec,113,2
9,Jonacatepec,114,2
11,Miacatlán,119,2
8,Jojutla,122,3


In [56]:
import plotly.express as px


prod_est_df = prod_est_df.rename(columns={
    'Nommunicipio': 'Municipio',
    'Volumenproduccion': 'Volumen de Producción'
})


# Crear el gráfico de barras
fig2 = px.bar(
    prod_est_df,
    x='Municipio',
    y='Volumen de Producción',
    hover_data={'Porcentaje': ':.2f'},
    text_auto='.2s',
    color="Volumen de Producción",
    color_continuous_scale=px.colors.sequential.Viridis,

)

# Actualizar el layout
fig2.update_layout(
    title="Producción de Limón en el estado de Morelos",
         title_font=dict(
        family="Arial, sans-serif",  # Fuente estética, puedes usar otras como 'Courier New', 'Times New Roman', etc.
        size=24,                    # Tamaño del título
        color="darkblue"            # Color del título
    ),
    xaxis_title="Municipio",  # Ahora es el eje X
    yaxis_title="Volumen de Producción (toneladas)",  # Ahora es el eje Y
    font=dict(size=8),
    yaxis=dict(tickformat=","),
    template="plotly_white"
)


fig2.update_traces(textfont_size=5, textangle=0, textposition="outside", cliponaxis=False)

# Mostrar el gráfico
fig2.show()

fig2.write_image("produccion_estatal.svg", engine="kaleido")
# Save the graph as an HTML file
fig2.write_html("produccion_estatal.html")

# Optionally, open the HTML file in the default web browser
import webbrowser
webbrowser.open("produccion_estatal.html")

True